## Hypothesis testing 

In class we learned about the different aspects of hypothesis testing and some of the theory behind the Z-test. 
Time to try this out in R.

### The data

An experiment was conducted to test the effect of two different drugs on an individual's duraction of sleep.
Forty patients were enrolled in this sleep study with 10 randomized to receive drug A, 10 randmized to recieve drug B, and 20 randomized to receive no drug at all. To account for differences in individuals each of the 20 particpants who recieved a drug were matched with a control patient. 

The difference in the number of hours slept by a patient who recieved a drug vs the matched control patient was recorded for all twenty pairs.

### From data to random variable

We can model the difference in the number of hours of sleep between a patient who recieved treatment and control with a random variable $Z$. This random variable needs to take values that are negative, times when the control patient slept longer than the treated patient, and values that are positive, times when the treated patient slept longer than the control patient.  

Lets define $Z \sim \mathcal{N}(\mu, \sigma^{2})$.

Further assume we know from previous experiment that $\sigma^{2} = 3$ and so our random variable $Z$ simplifies to 
$Z \sim \mathcal{N}(\mu, 3)$. 

### The hypothesis and the corresponding test

A statistical hypothesis is a pair of complementary statements about a parameter. 
In this case, our parameter of interest is $\mu$ the expected difference in the number of hours slept in patients who were treated vs control. 

One hypothesis of interest may be

\begin{align}
    \text{H}_{0} &: \mu = 0 \\ 
    \text{H}_{1} &: \mu > 0 \\ 
\end{align}

and we may wish to hold our Type I error---the probability that we conclude our parameter is in $H_{1}$ when truly our parameter is in $H_{0}$---to $\alpha=0.05$. 

To test this hypothesis, (ie to make a decision about whether $\mu$ belongs in $\text{H}_{0}$ or $\text{H}_{1}$) we can compute a Z-test. 
Lets do this by hand first and then use built-in functions in R. 

The Z test statistic is 

\begin{align}
    z = \frac{\overline{z} - \mu_{0}}{\sigma / \sqrt{N}}
\end{align}

where $\overline{z}$ is the sample mean of our random variable of interest and $\mu_{0}$ is the parameter value we assume under the null hypothesis.

Lets import the sleep data and compute $\overline{z}$.
The varible that measured the difference in number of hours of sleep is called ``extra``. 

In [2]:
d = read.csv("sleep.csv")
head(d)

,X,extra,group,ID
,<int>,<dbl>,<int>,<int>
1,1,0.7,1,1
2,2,-1.6,1,2
3,3,-0.2,1,3
4,4,-1.2,1,4
5,5,-0.1,1,5
6,6,3.4,1,6


In [5]:
zbar = mean(d$extra)

print(zbar)

[1] 1.54


We found that $\overline{z}$ is 1.54 hours and we can use this value to compute $z$ above. 

In [7]:
N = nrow(d)

z = (zbar - 0)/(sqrt(3/N))
print(z)

[1] 3.976263


The critical value for a Z-test with $\alpha=0.05$ is $c=1.96$. 
Because our test statistic is larger than 1.96 ($z > c$), then we conclude that $\mu \not \in \text{H}_{0}$---we reject the null hypothesis that the drug had no impact on sleep and accept the hypothesis that this drug significantly increases sleep ($\text{H}_{1}$).

We can also compute the **pvalue**, assuming our null hypothesis was true, the probability of a test statistic $z$ or "more extreme" could be generated by this distribution.
More extereme in our case are $\mu$ values greater than 0.

We need to compute $P( Z_{\text{test}} >z )$ where $Z_{\text{test}}$ is a r.v. that represents value of our test statistic under the null hypothesis. We know from class that

\begin{align}
    Z_{\text{test}} &\sim \mathcal{N}(0, 1) \\  
\end{align}

We need to compute $P( Z_{\text{test}} > 2.29 )$.
Lets use R. 

The `pnorm` function in R is equivalent to the cumulative density function for a random variable with a normal distribution. The `pnorm` function takes three arguments: the first argument is the value the CDF should evaluate, the second argument is the parameter value $\mu$, and the third argument is the parameter value $\sigma$. 

For example, define a random variable $X \sim \mathcal{N}(1,2)$. Then we can compute $F_{X}(1.5)$ in R as

In [34]:
cdfvalue = pnorm(1.5, 1, sqrt(2)) # why did we need the square root here? 
print(cdfvalue)

[1] 0.6381632


For our above problem we need to compute $P( Z_{\text{test}} > 2.29 )$ or $1 - F_{Z_{\text{test}}}(2.29)$

In [8]:
pvalue = 1 - pnorm(3.98, 0, 1)
print(pvalue)

[1] 3.445763e-05


There is a 0.00003 probability that if our null hypothesis was true then samples of our data could produce a test statistic like what we observed or values more extreme. Because this is unlikely then we would reject our null hypothesis in favor of our alternative hypothesis. 

The Z-test assumes that we know $\sigma^{2}$ exactly. 
An alternative test called the t-test assumes that we do **not** know $\sigma^{2}$ and must estimate this value from the data. 

A t-test can be run in R with a single command. 

In [36]:
t.test(d$extra,alternative="greater" )


	One Sample t-test

data:  d$extra
t = 3.413, df = 19, p-value = 0.001459
alternative hypothesis: true mean is greater than 0
95 percent confidence interval:
 0.7597797       Inf
sample estimates:
mean of x 
     1.54 


Here we find the mean of our variable of interest, the t-statistic (like the z-statistic), a statement of the alternative hypothesis, and a p-value.
The reason the t-test produces a different pvalue is because the t-statistic has a different probability distribution than the z-statisitc, and because our assumed exact $\sigma^{2}=3$ is different than what we would estimate from the data.

## Assignment

We learned about another test in class called the two-sample t-test which is used to test whether observed data could have been produced from random varibales with two different distributions. 

The dataset we want to explore is a dataset of highway accidents (documentation here = https://vincentarelbundock.github.io/Rdatasets/doc/carData/Highway1.html). 

The description of this data reads "The data comes from a unpublished master's paper by Carl Hoffstedt. They relate the automobile accident rate, in accidents per million vehicle miles to several potential terms. The data include 39 sections of large highways in the state of Minnesota in 1973. The goal of this analysis was to understand the impact of design variables, Acpts, Slim, Sig, and Shld that are under the control of the highway department, on accidents." and i have stored this dataset as "Highway1.csv"

1. Import the highway data and store it as ``D``
2. Seperate ``D`` into two datasets. The first dataset ``D1`` will be a roadway of type "PA" and the second dataset ``D2`` will be a roadway of type "MA".  
3. Use the documentation here https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/t.test to compare the rate of accidents per million vehicle miles between sections of road ways that are marked "PA" vs "MA" with a two-sample t-test.
4. Describe your results. 